In [1]:
# Dependencies and Setup
import pandas as pd

# Loading Files
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_csv)
student_data = pd.read_csv(student_csv)

# Combine the data into a single dataset.  
school_full = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_full.head(1)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635


In [3]:
### DISTRICT SUMMARY

#Number of Schools
schools_count = len(school_full["school_name"].value_counts())
school_count = len(school_data)
#print(school_count)

#Total Number of Students
stu_count = school_full["Student ID"].count()
student_count = "{:,}".format(len(student_data))
#print(student_count)

#Total Budget of the School District
total_budget = "${:,.2f}".format(school_data["budget"].sum())
#print(total_budget)

#Calculating Average Math and Reading Scores
avg_math = school_full["math_score"].mean()
#print(avg_math)

avg_read = school_full["reading_score"].mean()
#print(avg_read)

#Percent of Students Passing Math (Score of 70%+)
count_pass_math = sum(x >=70 for x in school_full["math_score"])
perc_pass_math = "{:.2%}".format(count_pass_math/stu_count)
#print(perc_pass_math)

#Percent of Students Passing Reading (Score of 70%+)
count_pass_read = sum(x >=70 for x in school_full["reading_score"])
perc_pass_read = "{:.2%}".format(count_pass_read/stu_count)
#print(perc_pass_read)

#Percent of Overall Students Passing (where both math and reading scores are greater than 70)
overall_pass = school_full.loc[(school_full["math_score"]>=70)&(school_full["reading_score"]>=70)]
perc_overall_pass = "{:.2%}".format((len(overall_pass)/stu_count))
#print(perc_overall_pass)

#Creating Dictionary of Fields to be Included in Dataframe
dist_data = [{"Total Schools": school_count,"Total Students": stu_count,"Total Budget": total_budget,
              "Average Math Score": avg_math,"Average Reading Score": avg_read,
             "% Passing Math": perc_pass_math,"% Passing Reading": perc_pass_read,
             "% Overall Passing": perc_overall_pass}]

In [4]:
### DISTRICT SUMMARY DATAFRAME
district_df = pd.DataFrame(dist_data)
display(district_df)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


In [5]:
### SCHOOL SUMMARY

#Grouping full dataset by school name
school = school_full.groupby(['school_name'])

#Sorting school name list so it will align when referenced later
school_data.sort_values("school_name",axis=0,ascending=True,inplace=True)

#Pulling fields from school data file that are only unique at a school level

names = list(school_data["school_name"])
#print(names)

types = list(school_data["type"])
#print(types)

school_budget = list(school_data["budget"])
#print(school_budget)

#Getting total number of students that attend each school
count = list(school["school_name"].size())
#print(count)

#Calculating Per Student Budget for Each School
budget_per_stud = []
for i in range(len(names)):
    budg = (school_budget[i]/count[i])
    budget_per_stud.append(budg)
#print(budget_per_stud)


#Calculating Average Math and Reading Scores For Each School

avg_school_math = list(school["math_score"].mean())
#print(avg_school_math)

avg_school_read = list(school["reading_score"].mean())
#print(avg_school_read)


#Count of Students Passing Math at Each School (Score of 70%+)
school_count_pass_math = school.apply(lambda x: x[x['math_score'] >= 70 ]['Student ID'].count())
math = list(school_count_pass_math)
#print(math)

#Using Count of Students Passing Math and Number of Students at Each School to Determine % Passing
school_perc_math = []
for i in range(len(count)):
    perc = (math[i]/count[i])
    school_perc_math.append(perc)
#print(school_perc_math)

#Count of Students Passing Reading at Each School (Score of 70%+)
school_count_pass_read = school.apply(lambda x: x[x['reading_score'] >= 70]['Student ID'].count())
read = list(school_count_pass_read)
#print(read)

#Using Count of Students Passing Reading and Number of Students at Each School to Determine % Passing
school_perc_read = []
for i in range(len(count)):
    perc = (read[i]/count[i])
    school_perc_read.append(perc)
#print(school_perc_read)

#Determining Overall % of Students Passing
school_pass = []
school_pass_overall = school.apply(lambda x: x[(x['math_score'] >= 70)&(x['reading_score'] >= 70)]['Student ID'].count())
overall = list(school_pass_overall)
for i in range(len(count)):
    perc = (overall[i]/count[i])
    school_pass.append(perc)
#print(school_pass)


school_tbl = {"School Type":types,"Total Students": count,"Total School Budget": school_budget,
              "Per Student Budget": budget_per_stud,"Average Math Score": avg_school_math,"Average Reading Score": avg_school_read,
             "% Passing Math": school_perc_math,"% Passing Reading": school_perc_read,
             "% Overall Passing": school_pass}

#school_tbl

In [6]:
### SCHOOLS SUMMARY DATAFRAME
school_df = pd.DataFrame(school_tbl)
#Adding the school names as the index for the dataframe
school_df= school_df.set_index(pd.Index(names))
school_df_clean = school_df.style.format(precision=4,formatter={"Total Students":"{:,}","Total School Budget":"${:,.2f}","Per Student Budget":"${:,.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})
display(school_df_clean)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0484,81.0340,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.0619,83.9758,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7118,81.1580,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1026,80.7463,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.3515,83.8168,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.2898,80.9344,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.8033,83.8150,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6294,81.1827,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.0725,80.9664,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.59%,95.95%,90.54%


In [7]:
### TOP PERFORMING SCHOOLS (BY % OVERALL PASSING)
#Sort Dataframe descending so that top schools based on overall passing are shown at the top of the dataframe
top_5_schools = school_df.sort_values("% Overall Passing", ascending=False).head(5)
top_5_schools.style.format(precision=4,formatter={"Total School Budget":"${:,.2f}","Per Student Budget":"${:,.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4183,83.8489,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.2742,83.9895,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.59%,95.95%,90.54%


In [8]:
### BOTTOM PERFORMING SCHOOLS (BY % OVERALL PASSING)
#Same as above but instead set ascending = True so worst schools were at the top
worst_5_schools = school_df.sort_values("% Overall Passing", ascending=True).head(5)
worst_5_schools.style.format(precision=4,formatter={"Total School Budget":"${:,.2f}","Per Student Budget":"${:,.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8427,80.7447,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.1580,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.2898,80.9344,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,66.06%,81.22%,53.54%


In [9]:
### MATH SCORES BY GRADE
#Group by both School Name and Grade
school_grade = school_full.groupby(["school_name","grade"]).mean().sort_values(["school_name","grade"],ascending=True)

school_grade_math = pd.DataFrame(school_grade["math_score"])
#display(school_grade_math)

#Used this to check grade groups that are created below
check_math = school_grade_math.reset_index(level='grade')
#display(check_math)

#Creating series containing math scores for each grade

ninth_math = check_math[check_math["grade"]=="9th"]
ninth_math = list(ninth_math.loc[:,"math_score"])
#print(ninth_math)

tenth_math = check_math[check_math["grade"]=="10th"]
tenth_math = list(tenth_math.loc[:,"math_score"])
#print(tenth_math)

jr_math = check_math[check_math["grade"]=="11th"]
jr_math = list(jr_math.loc[:,"math_score"])
#print(jr_math)

sr_math = check_math[check_math["grade"]=="12th"]
sr_math = list(sr_math.loc[:,"math_score"])
#print(sr_math)

math_grade_data = {"9th":ninth_math, "10th":tenth_math,
                   "11th": jr_math, "12th": sr_math}

In [10]:
### MATH SCORES BY GRADE SUMMARY DATAFRAME
math_scores_df = pd.DataFrame(math_grade_data)
math_scores_df = math_scores_df.set_index(pd.Index(names))
display(math_scores_df)

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [11]:
### READING SCORE BY GRADE
school_grade_read = pd.DataFrame(school_grade["reading_score"])
#display(school_grade_read)

#Used this to check grade groups that are created below
check_read = school_grade_read.reset_index(level='grade')
#display(check_read)

#Creating series containing reading scores for each grade

ninth_read = check_read[check_read["grade"]=="9th"]
ninth_read = list(ninth_read.loc[:,"reading_score"])
#print(ninth_read)

tenth_read = check_read[check_read["grade"]=="10th"]
tenth_read = list(tenth_read.loc[:,"reading_score"])
#print(tenth_read)

jr_read = check_read[check_read["grade"]=="11th"]
jr_read = list(jr_read.loc[:,"reading_score"])
#print(jr_read)

sr_read = check_read[check_read["grade"]=="12th"]
sr_read = list(sr_read.loc[:,"reading_score"])
#print(sr_read)

read_grade_data = {"9th":ninth_read, "10th":tenth_read,
                   "11th": jr_read, "12th": sr_read}

In [12]:
### READING SCORES BY GRADE SUMMARY DATAFRAME
read_scores_df = pd.DataFrame(read_grade_data)
read_scores_df = read_scores_df.set_index(pd.Index(names))
display(read_scores_df)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [13]:
import numpy as np

In [14]:
### SCORES BY SCHOOL SPENDING
#Creating bins based on per student budget
spending_groups = ["<$585","$585-630","$630-645",">$645"]
bins = [0,585,630,645,np.inf]

#Creating Column to Store Spending Group Categories
school_df["Spending Ranges (Per Student)"] = pd.cut(school_df["Per Student Budget"],bins,labels = spending_groups)
#display(school_df)

spend_df = school_df.groupby(["Spending Ranges (Per Student)"]).mean()
spend_summary = spend_df.drop(['Total Students', 'Total School Budget', 'Per Student Budget'],axis=1)
spend_summary.style.format(formatter={"Average Math Score":"{:.2f}","Average Reading Score":"{:.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
>$645,77.00,81.03,66.16%,81.13%,53.53%


In [15]:
### SCORES BY SCHOOL SIZE
#Creating bins to separate school sizes
size_groups = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
bins = [0,1000,2000,np.inf]

#Creating Column to Save School Size Bins
school_df["School Size"] = pd.cut(school_df["Total Students"],bins,labels = size_groups)
#display(school_df)

size_df = school_df.groupby(["School Size"]).mean()
size_summary = size_df.drop(['Total Students', 'Total School Budget', 'Per Student Budget'],axis=1)
size_summary.style.format(formatter={"Average Math Score":"{:.2f}","Average Reading Score":"{:.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


In [16]:
### SCORES BY SCHOOL TYPE

type_df = school_df.groupby(["School Type"]).mean()

#Did not drop Total Students or Per Student Budget Column columns yet because it is used to support analysis
type_summary = type_df.drop(['Total School Budget'],axis=1)
type_summary.style.format(formatter={"Per Student Budget": "${:.2f}","Average Math Score":"{:.2f}","Average Reading Score":"{:.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})

,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,,,
Charter,1524.250000,$599.50,83.47,83.90,93.62%,96.59%,90.43%
District,3853.714286,$643.57,76.96,80.97,66.55%,80.80%,53.67%


In [17]:
#SCORES BY SCHOOL TYPE (FINAL TABLE)
type_final = type_summary.drop(['Total Students','Per Student Budget'], axis=1)
type_final.style.format(formatter={"Per Student Budget": "${:.2f}","Average Math Score":"{:.2f}","Average Reading Score":"{:.2f}","% Passing Math":"{:.2%}","% Passing Reading":"{:.2%}","% Overall Passing":"{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


In [18]:
#Looking at Math Scores Across Grades and School Type
math_grades_type = math_scores_df
math_grades_type['School Type'] = school_df['School Type']
display(math_grades_type)

,9th,10th,11th,12th,School Type
Bailey High School,77.083676,76.996772,77.515588,76.492218,District
Cabrera High School,83.094697,83.154506,82.765560,83.277487,Charter
Figueroa High School,76.403037,76.539974,76.884344,77.151369,District
Ford High School,77.361345,77.672316,76.918058,76.179963,District
Griffin High School,82.044010,84.229064,83.842105,83.356164,Charter
Hernandez High School,77.438495,77.337408,77.136029,77.186567,District
Holden High School,83.787402,83.429825,85.000000,82.855422,Charter
Huang High School,77.027251,75.908735,76.446602,77.225641,District
Johnson High School,77.187857,76.691117,77.491653,76.863248,District
Pena High School,83.625455,83.372000,84.328125,84.121547,Charter
